# Muslytics EDA
## Setup
Run the contents of the following cells to load the relevant packages and data.

The second cell will read the tracks table in the muslytics database into dataframe `df`. You will be prompted for a password.

In [1]:
import getpass

from muslytics.DatabaseUtils import connect, Session
from muslytics.Utils import Track

from bokeh.charts import BoxPlot, Histogram
from bokeh.io import output_notebook, push_notebook, show
from bokeh.layouts import row, column
from bokeh.models import CategoricalColorMapper, ColumnDataSource, HoverTool
from bokeh.models.widgets import Tabs, Panel
from bokeh.palettes import Spectral
from bokeh.plotting import figure
from ipywidgets import interact
import numpy as np
import pandas as pd

output_notebook()

Loading BokehJS ...

In [2]:
db = connect('root', getpass.getpass(), 'localhost', 'muslytics')
select = Session().query(Track)
results = db.execute(select.statement).fetchall()
df = pd.read_sql(select.statement, db, index_col='id')

print('Read {num} tracks'.format(num=len(df)))

········
Read 1690 tracks


## Shaping
Run this cell to shape the dataframe. There will be two dataframes available for use afterwards: `df` and `dummy_df`. The results of the shaping on both are described below.

- `df`, `dummy_df`: Drop the `spotify_id` column
- `dummy_df`: Dummify the genre series into columns prefxied by `'genre_'` that represent the genre of the track. For example, a pop song would only have a 1 in the `genre_pop` column, and 0s in every other column. One genre level will be dropped and is thus represented by a track that has 0 in every `genre_X` column.

In [3]:
df = df.drop('spotify_id', 1)
dummy_df = pd.get_dummies(df, prefix='genre', columns=['genre'], drop_first=True)

## Summary Stats
Run the following cell to see the summary statistics for the tracks table.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1690 entries, 2545 to 8803
Data columns (total 21 columns):
name                1690 non-null object
artists             1690 non-null object
genre               1690 non-null object
plays               1690 non-null int64
rating              1690 non-null float64
loved               1690 non-null bool
popularity          1690 non-null int64
acousticness        1690 non-null float64
danceability        1689 non-null float64
duration_ms         1690 non-null int64
energy              1690 non-null float64
instrumentalness    1690 non-null float64
key                 1690 non-null int64
liveness            1690 non-null float64
loudness            1690 non-null float64
mode                1690 non-null int64
speechiness         1689 non-null float64
tempo               1690 non-null float64
time_signature      1689 non-null float64
valence             1689 non-null float64
year                1690 non-null int64
dtypes: bool(1), float64(1

In [5]:
df.loc[df.isnull().any(axis=1)]

,name,artists,genre,plays,rating,loved,popularity,acousticness,danceability,duration_ms,...,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,year
id,,,,,,,,,,,,,,,,,,,,,
3375,Horrible Theme,Instrumental,soundtrack,8,3.0,False,0,0.568,NaN,9872,...,0.977,9,0.237,-10.925,0,NaN,0.0,NaN,NaN,2008


In [6]:
df.describe()

,plays,rating,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,year
count,1690.000000,1690.000000,1690.000000,1690.000000,1689.000000,1690.000000,1690.000000,1690.000000,1690.000000,1690.000000,1690.000000,1690.000000,1689.000000,1690.000000,1689.000000,1689.000000,1690.000000
mean,201.611243,3.655030,47.363314,0.216919,0.551033,217320.015976,0.685875,0.057874,5.105917,0.173761,-6.199934,0.798225,0.060370,125.187776,3.932504,0.484611,2011.813018
std,633.636509,0.837927,15.558726,0.263276,0.128896,48296.096838,0.203154,0.208527,3.481155,0.129637,2.845735,0.401444,0.072681,29.314516,0.302346,0.230956,3.915395
min,0.000000,1.000000,0.000000,0.000022,0.097200,9872.000000,0.047300,0.000000,0.000000,0.021000,-28.663000,0.000000,0.022500,0.000000,1.000000,0.027600,1975.000000
25%,12.000000,3.000000,37.000000,0.018900,0.478000,193497.500000,0.560250,0.000000,2.000000,0.093550,-7.270750,1.000000,0.032000,102.031000,4.000000,0.303000,2010.000000
50%,35.000000,4.000000,48.000000,0.092900,0.558000,215500.000000,0.733000,0.000000,5.000000,0.121000,-5.576500,1.000000,0.039700,123.963000,4.000000,0.485000,2013.000000
75%,103.000000,4.000000,60.000000,0.330500,0.636000,239894.250000,0.844750,0.000021,8.000000,0.213000,-4.371750,1.000000,0.055200,145.991750,4.000000,0.660000,2015.000000
max,12012.000000,5.000000,85.000000,0.986000,0.965000,536080.000000,0.992000,0.990000,11.000000,0.952000,-0.521000,1.000000,0.852000,207.914000,5.000000,0.965000,2016.000000


### Play Count Analysis
Some basic EDA by play count.

In [7]:
def get_bounds(df):
    col_vals = df.plays
    s3 = 3*np.std(col_vals)
    col_mean = np.mean(col_vals)
    lower = col_mean - s3
    upper = col_mean + s3
    print("\tThe mean plays is {mean}. Outliers are < {x} and > {y}".format(mean=col_mean,
                                                                            x=lower,
                                                                            y=upper))
    return (col_mean, lower, upper)

def find_outliers(df, lower, upper):
    outliers = df.loc[df['plays'] > upper]
    print("\tThere are {x} outliers by play count".format(x=len(outliers)))
    return outliers

def drop_outliers(df, outliers):
    dropped = df.drop(outliers.index)
    print("Removing {x} outliers. {y} tracks left.".format(x=len(outliers), y=len(dropped)))
    return dropped

In [9]:
print("Starting with all {x} songs ...".format(x=len(df)))
mean, lower, upper = get_bounds(df)
outliers = find_outliers(df, lower, upper)
no_outliers = []
current_df = df

while len(outliers) > 0:
    no_outlier_df = drop_outliers(current_df, outliers)
    no_outliers.append(no_outlier_df)
    mean, lower, upper = get_bounds(no_outlier_df)
    outliers = find_outliers(no_outlier_df, lower, upper)
    current_df = no_outlier_df

no_outlier_df = no_outliers[-1]

Starting with all 1690 songs ...
	The mean plays is 201.611242604. Outliers are < -1698.73580142 and > 2101.95828663
	There are 34 outliers by play count
Removing 34 outliers. 1656 tracks left.
	The mean plays is 130.794082126. Outliers are < -774.48578927 and > 1036.07395352
	There are 55 outliers by play count
Removing 55 outliers. 1601 tracks left.
	The mean plays is 82.3285446596. Outliers are < -343.484448305 and > 508.141537625
	There are 47 outliers by play count
Removing 47 outliers. 1554 tracks left.
	The mean plays is 63.2606177606. Outliers are < -197.573344535 and > 324.094580056
	There are 47 outliers by play count
Removing 47 outliers. 1507 tracks left.
	The mean plays is 52.7266091573. Outliers are < -137.879552521 and > 243.332770836
	There are 37 outliers by play count
Removing 37 outliers. 1470 tracks left.
	The mean plays is 46.9217687075. Outliers are < -110.508838035 and > 204.35237545
	There are 31 outliers by play count
Removing 31 outliers. 1439 tracks left.
	Th

In [10]:
no_outlier_df.describe()

,plays,rating,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,year
count,1334.000000,1334.000000,1334.000000,1334.000000,1333.000000,1334.000000,1334.000000,1334.000000,1334.000000,1334.000000,1334.000000,1334.000000,1333.000000,1334.000000,1333.000000,1333.000000,1334.000000
mean,33.532984,3.496252,48.359070,0.215745,0.551779,215944.292354,0.689513,0.061198,5.151424,0.171910,-6.188160,0.793853,0.062115,125.771738,3.929482,0.484980,2012.182159
std,31.572492,0.830532,15.777642,0.261651,0.126618,48688.141477,0.200837,0.213366,3.482397,0.127623,2.771121,0.404689,0.078870,29.899353,0.318795,0.227169,3.845939
min,0.000000,1.000000,0.000000,0.000022,0.124000,9872.000000,0.051600,0.000000,0.000000,0.021000,-25.889000,0.000000,0.022500,0.000000,1.000000,0.029300,1975.000000
25%,8.000000,3.000000,37.000000,0.018900,0.479000,192830.000000,0.563250,0.000000,2.000000,0.093550,-7.289500,1.000000,0.031900,102.045000,4.000000,0.308000,2011.000000
50%,25.000000,4.000000,49.000000,0.092200,0.556000,213773.000000,0.737000,0.000000,5.000000,0.121000,-5.575000,1.000000,0.039800,124.944500,4.000000,0.489000,2013.000000
75%,50.000000,4.000000,61.000000,0.325000,0.631000,238033.250000,0.846000,0.000025,8.000000,0.208750,-4.389500,1.000000,0.054800,147.964250,4.000000,0.659000,2015.000000
max,126.000000,5.000000,85.000000,0.986000,0.965000,536080.000000,0.992000,0.990000,11.000000,0.952000,-0.521000,1.000000,0.852000,207.914000,5.000000,0.965000,2016.000000


In [11]:
def make_plays_rating_hist(df, title):
    plays = Histogram(df, 'plays', bins=10, title='Play Count Distribution ({x})'.format(x=title))
    ratings = Histogram(df, 'rating', bins=5, title='Rating Distribution ({x})'.format(x=title))
    return (plays, ratings)

def make_plays_boxplot(df, title):
    return BoxPlot(df, values='plays', label='genre', color='genre', title='Plays by Genre ({x})'.format(x=title))

In [12]:
all_row = row(*make_plays_rating_hist(df, 'All'))
no_outlier_row = row(*make_plays_rating_hist(no_outlier_df, 'Removed Outliers'))
layout = column(all_row, no_outlier_row)
show(layout)

In [13]:
all_row = make_plays_boxplot(df, 'All')
no_outlier_row = make_plays_boxplot(no_outlier_df, 'Removed Outliers')
layout = column(all_row, no_outlier_row)
show(layout)

In [18]:
genres = no_outlier_df.genre.unique()
panels = []
for genre in genres:
    no_outlier_genre_df = no_outlier_df.loc[no_outlier_df.genre == genre]
    plays, ratings = make_plays_rating_hist(no_outlier_genre_df, '{x}: Removed Outliers'.format(x=genre.title()))
    play_box = make_plays_boxplot(no_outlier_genre_df, '{x}: Removed Outliers'.format(x=genre.title()))
    panels.append(Panel(child=column(row(plays, play_box), ratings), title="{g} ({x})".format(g=genre.title(),
                                                                                             x=len(no_outlier_genre_df))))
    
tabs = Tabs(tabs=panels)
show(tabs)

## Feature Correlation
- Run the first cell to perform necessary setup.
- The second cell outputs a scatter plot showing the correlation between two features.
- The third cell gives dropdowns to select which data the plot should show.

In [15]:
spectral11 = Spectral[10]
color_map = { genre : spectral11[i % len(spectral11)] for i, genre in enumerate(df.genre.unique())}
mapper = CategoricalColorMapper(factors=color_map.keys(), palette=color_map.values())

hover = HoverTool(tooltips=[
        ('name', '@name'),
        ('plays', '@plays'),
        ('rating', '@rating'),
        ('genre', '@genre')
    ])

source = ColumnDataSource(data={'x': no_outlier_df['valence'],
                                'y': no_outlier_df['acousticness'],
                                'name': no_outlier_df['name'],
                                'genre': no_outlier_df['genre'],
                                'rating': no_outlier_df['rating'],
                                'plays': no_outlier_df['plays']})

def update(genre='all', x='valence', y='acousticness'):
    if genre == 'all':
        data = no_outlier_df
    else:
        data = no_outlier_df.loc[no_outlier_df['genre'] == genre]
        
    source.data['x'] = data[x]
    source.data['y'] = data[y]
    plot.title.text = '{x} by {y} ({g})'.format(x=x.title(), y=y.title(), g=genre.title())
    plot.xaxis.axis_label = x.title()
    plot.yaxis.axis_label = y.title()
    push_notebook(handle=interhandle)

plot = figure(x_axis_label='valence',
              y_axis_label='acousticness',
              tools=[hover, 'pan', 'wheel_zoom', 'save'],
              title='Valence vs. Acousticness (All)', plot_height=400, plot_width=400)
plot.scatter('x', 'y',
          source=source,
          fill_alpha=0.6,
          size=5
         )
options = ['popularity', 'acousticness', 'danceability', 'duration_ms',
           'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness',
           'tempo', 'valence', 'rating', 'plays']
genres = ['all']
genres.extend(df.genre.unique())

In [16]:
interhandle = show(plot, notebook_handle=True)

In [17]:
_ = interact(update, genre=genres, x=options, y=options)